In [2]:
import shutil
import zipfile
import cv2
import os
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import random
import numpy as np
from ultralytics import YOLO
from PIL import Image

In [3]:

# Input
input_dir = 'D:\yolo\yolov8_helmet_detection_main\yolov8_helmet_detection_main\Data'
images_dir = os.path.join(input_dir, "images")
annotations_dir = os.path.join(input_dir, "annotations")

# Output
working_dir = 'D:\\yolo\yolov8_helmet_detection_main\\yolov8_helmet_detection_main\\working\\'
labels_dir = os.path.join(working_dir, "labels")
train_img_dir = os.path.join(working_dir, "train", "images")
train_labels_dir = os.path.join(working_dir, "train", "labels")
val_img_dir = os.path.join(working_dir, "val", "images")
val_labels_dir = os.path.join(working_dir, "val", "labels")
models_dir = os.path.join(working_dir, "models")
predict_dir = os.path.join(working_dir, "predict")

In [4]:
def parse_xml(xml_file):
    '''
    Parse the XML file and extract image information and bounding boxes

    Args:
        xml_file: str: path to the XML file

    Return:
        image_name: str: name of the image file
        image_shape: tuple: shape of the image
        labels_and_bboxes: list: list of tuples containing labels and bounding boxes
    '''

    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract image information
    image_name = root.find('filename').text
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)
    depth = int(root.find('size/depth').text)
    image_shape = width, height, depth

    labels_and_bboxes = []

    # Loop through each object in the XML
    for obj in root.findall('object'):
        # Extract label and bounding box coordinates for each object
        label = obj.find('name').text
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        # Append label and bounding box to the list
        labels_and_bboxes.append((label, (xmin, ymin, xmax, ymax)))

    return image_name, image_shape, labels_and_bboxes


In [5]:
def draw_bounding_boxes(img_file, labels_and_bboxes):
    '''
    Draw bounding boxes on the image

    Args:
        img_file: str: path to the image file
        labels_and_bboxes: list: list of tuples containing labels and bounding boxes
    '''
    # Load the image
    image = cv2.cvtColor(cv2.imread(img_file), cv2.COLOR_BGR2RGB)

    # Draw bounding boxes on the image
    for label, bbox in labels_and_bboxes:
        xmin, ymin, xmax, ymax = bbox
        rgb_color = (0, 255, 0) if label == 'With Helmet' else (255, 0, 0)

        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), rgb_color, 2)
        cv2.putText(image, label, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, rgb_color, 1)

    # Display the image with bounding boxes
    plt.axis(False)
    plt.title(os.path.split(img_file)[-1], y=-0.1)
    plt.imshow(image)


In [6]:
def get_random_img_xml():
    '''
    Get a random image and its corresponding XML file

    Return:
        img_file: str: path to the image file
        xml_file: str: path to the XML file
    '''

    # pick a random image from the dataset
    img_name = random.choice(os.listdir(images_dir))
    img_file = os.path.join(images_dir, img_name)
    xml_file = os.path.join(annotations_dir, img_name[:-4]+'.xml')

    return img_file, xml_file

In [7]:
img_file, xml_file = get_random_img_xml()
image_name, image_shape, labels_and_bboxes = parse_xml(xml_file)
# draw_bounding_boxes(img_file, labels_and_bboxes)

In [8]:
def create_labels(xml_dir, labels_dir):
    '''
    Create labels for each image in the dataset. Ignore image if invalid bboxes (coordinates out of image shape)

    Args:
        xml_dir: str: path to the directory containing the annotation xml files
        labels_dir: str: path to the directory where the labels will be saved
    '''

    # browse through annotation xml files and extract the class and bounding box coordinates
    os.makedirs(labels_dir, exist_ok=True)

    annotations = [file for file in os.listdir(xml_dir) if file.lower().endswith('.xml')]

    count = 0
    ignored = 0

    for xml_file in annotations:

        image_name, image_shape, labels_and_bboxes = parse_xml(os.path.join(xml_dir, xml_file))

        # save label and bbox to a text file with same name than image file
        txt_file = os.path.join(labels_dir, xml_file.replace('.xml', '.txt'))

        file_corrupt = False

        with open(txt_file, 'w') as f:

            for label, bbox in labels_and_bboxes:

                # get label
                label = 1 if label == 'With Helmet' else 0

                # compute bounding box center, width and height from bbox coordinates
                x_center = (bbox[0] + bbox[2]) / 2
                y_center = (bbox[1] + bbox[3]) / 2
                width = bbox[2] - bbox[0]
                height = bbox[3] - bbox[1]

                # normalize all values between 0 and 1
                x_center /= image_shape[0]
                y_center /= image_shape[1]
                width /= image_shape[0]
                height /= image_shape[1]

                # check if values are within the range 0 and 1
                if x_center > 1 or y_center > 1 or width > 1 or height > 1:
                    file_corrupt = True
                    break

                f.write(f"{label} {x_center} {y_center} {width} {height}\n")

        # delete corrupted files (values don't make any sense)
        if file_corrupt:
            ignored += 1
            f.close()
            os.remove(txt_file)
            continue

        print(f"\rImage: {image_name}     ", end='', flush=True)
        count += 1


    print(f"\n>> {count} labels created | {ignored} images ignored")


In [9]:
create_labels(annotations_dir, labels_dir)

Image: BikesHelmets99.png      
>> 749 labels created | 15 images ignored


In [10]:
def create_train_val_split():
    '''
    Create a train and val split of the images/labels.
    '''

    # make sure target dirs exist
    for dir in [train_img_dir, val_img_dir, train_labels_dir, val_labels_dir]:
        os.makedirs(dir, exist_ok=True)

    # copy images randomly to train and val folders using 80/20 split
    images = [img[:-4] for img in os.listdir(images_dir)]
    random.shuffle(images)
    split = int(0.8 * len(images))

    count_total = len(images)
    count_train = 0
    count_val = 0
    count_ignored = 0

    for i in range(len(images)):

        # check if label exists (some images are corrupted and don't have a label file)
        if not os.path.exists(os.path.join(labels_dir, f"{images[i]}.txt")):
            count_ignored += 1
            count_total -= 1
            continue

        if i < split:
            shutil.copy(os.path.join(images_dir, f"{images[i]}.png"), train_img_dir)
            shutil.copy(os.path.join(labels_dir, f"{images[i]}.txt"), train_labels_dir)
            count_train += 1
        else:
            shutil.copy(os.path.join(images_dir, f"{images[i]}.png"), val_img_dir)
            shutil.copy(os.path.join(labels_dir, f"{images[i]}.txt"), val_labels_dir)
            count_val += 1

        count_total -= 1

        print(f"\rImages: {count_total} >> Train: {count_train} | Val: {count_val} | Ignored: {count_ignored}     ", end='', flush=True)


In [11]:
create_train_val_split()

Images: 0 >> Train: 599 | Val: 150 | Ignored: 15      

In [12]:
config_file_path = os.path.join(working_dir, 'config.yaml')

config_file_contents = f'''path: {working_dir}  # root dir
train: train/images  # train dir
val: val/images  # val dir

# Classes
names:
  0: without helmet
  1: with helmet
'''

with open(config_file_path, 'w') as f:
    f.write(config_file_contents)

print(f"Config file written successfully at location {config_file_path}")

Config file written successfully at location D:\yolo\yolov8_helmet_detection_main\yolov8_helmet_detection_main\working\config.yaml
